In [64]:
# Imports of needed libraries
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash_bootstrap_templates import load_figure_template

In [65]:
# Df All Athletes
athlete_events = pd.read_csv("../data/final_df.csv")

# Df Swedish Athletes
sweden_athletes = pd.DataFrame(athlete_events[athlete_events["NOC"] == "SWE"])

gender_ratios = pd.read_csv("../data/gender_ratios.csv")

C:\Users\yonis\AppData\Local\Temp\ipykernel_2976\1946786575.py:2: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [66]:
participants_medals= athlete_events.groupby(['Year','Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_start = participants_medals.groupby(['Country','Continent' , 
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport = athlete_events.groupby(['Year', 'Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})


In [67]:
participants_medals_sport = athlete_events.groupby(['Year','Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_sport_total = participants_medals_sport.groupby(['Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport_year = participants_medals_sport.groupby(['Year','Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

medal_distribution = athlete_events.groupby(['Year','Country', 'Sport', 'Medal']).size().reset_index(name="Number of Medals")



In [85]:
medal_distribution_sweden = medal_distribution.query('Country == "Sweden"')



In [72]:

participants_medals_season = athlete_events.groupby(['Year','Season','Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})


participants_medals_season_start= participants_medals_season.groupby(['Country','Continent' , 
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

In [69]:
participants_medals_season.query('Season == "Summer"')

,Year,Season,Country,Continent,Country_latitude,Country_longitude,Continent_latitude,Continent_longitude,Name,Medal
0,1896,Summer,Australia,Oceania,-25.274398,133.775136,-14.2350,141.2551,1,3
1,1896,Summer,Austria,Europe,47.516231,14.550072,54.5260,15.2551,3,5
2,1896,Summer,Denmark,Europe,56.263920,9.501785,54.5260,15.2551,3,6
3,1896,Summer,France,Europe,46.227638,2.213749,54.5260,15.2551,12,11
4,1896,Summer,Germany,Europe,51.165691,10.451526,54.5260,15.2551,19,32
...,...,...,...,...,...,...,...,...,...,...
3496,2016,Summer,Vanuatu,Oceania,-15.376706,166.959158,-14.2350,141.2551,4,0
3497,2016,Summer,Venezuela,South America,6.423750,-66.589730,-14.2350,-59.2551,85,3
3498,2016,Summer,Yemen,Asia,15.552727,48.516388,34.0479,100.6197,3,0
3499,2016,Summer,Zambia,Africa,-13.133897,27.849332,7.1881,21.0938,7,0


In [104]:
# Loading template for graphs
load_figure_template("quartz")

# This initializes a Dash web application, 
# applies the "QUARTZ" Bootstrap theme for styling, 
# and includes a viewport meta tag to ensure proper rendering on various devices.
app = Dash(__name__, 
           external_stylesheets=[dbc.themes.QUARTZ],
               meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

# Needs to be included for deploying on render
server = app.server

# Layout for App
app.layout = html.Div([
    dbc.Row(
        [
            dbc.Col(html.H2("Olympic Games Achievements 1896-2016", className="text-center text-primary")),
        ],
        className="mb-3 mt-3", # Adding marginal bottom and top
    ),
    
    dbc.Row([
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_left', 
                    className='ml-3 mr-3 mb-1 mt-3 text-info', 
                    options=["Medals", "Gender ratio"], 
                    placeholder='Select to see participants with relation to medals or gender ratio'),
                    dcc.Graph(id="graph_1_left",    
                    figure={}),
                ], xs=12, sm=11, md=10, lg=5, width='auto'),
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_right', 
                    className='ml-3 mr-3 mb-1 mt-3 text-info', 
                    options=["Country", "Sport"], 
                    placeholder='Select to sort by country or sport'), 
                    dcc.Graph(id="graph_1_right",    
                    figure={})],
                    xs=12, sm=11, md=10, lg=5),
    ], justify='evenly'),

    dbc.Row([
            dcc.Dropdown(id='year_dropdown', className='text-info mt-2 mb-1',
                        multi=True, 
                        options=[years for years in sorted(athlete_events['Year'].unique())], 
                        placeholder='Select Year',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},
            ),
            dcc.Dropdown(id='sport_dropdown', className='text-info mt-2 mb-1',
                        multi=True, 
                        options=[sports for sports in sorted(athlete_events['Sport'].unique())], 
                        placeholder='Select Sport',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},
            ),

    ], justify='center', class_name="ml-50 mr-50"), 
        dbc.Row([
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_down_left', 
                    className='ml-3 mr-3 mb-1 mt-1 text-info', 
                    options=["Summer", "Winter"], 
                    placeholder='Select to sort according to Season'),
                    dcc.Graph(id="graph_1_down_left",    
                    figure={}),
                ], xs=12, sm=11, md=10, lg=5, width='auto'),
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_down_right', 
                    className='ml-3 mr-3 mb-1 mt-1 text-info', 
                    options=["Sports", "Medals"], 
                    placeholder='Select to sort by Sports or Medals'), 
                    dcc.Graph(id="graph_1_down_right",    
                    figure={})],
                    xs=12, sm=11, md=10, lg=5),
    ], justify='evenly'),


dbc.Row([
        dbc.Button("Reset", id='reset-button', className='float-left', n_clicks=0), # Callback to reset page
        dcc.Link("Contributors", href="https://github.com/DeerBay/OS-Project/graphs/contributors", target="_blank"), # "_blank": Opens the linked document in a new tab or window.
    ],style={"margin-top": "20px", "text-align": "center"})

])


@callback(
    Output("graph_1_left", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value"),
    Input("country_dropdown_left", "value")
)
def figure_one(years, sports, sort):
    if sort == "Medals" or sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760, hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1.2, opacity=0.5,
            title="Size according to count of participants")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = participants_medals.query("Year==@years")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1,
            title="Size according to count of participants")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800,width= 760, hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport_year.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants")
    elif sort =="Gender ratio":
        df = gender_ratios
        fig = px.scatter_mapbox(df, animation_frame = "Year", lat="Country_latitude",
                lon="Country_longitude", size="Count", color="Ratio", 
                height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
                center=dict(lat=0, lon=0), zoom=1, title="Gender ratios over the years")

    fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)
    
    return fig



@callback(
    Output("graph_1_right", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value"),
    Input("country_dropdown_right", "value"),

)
def figure_two(years, sports, sort):
    if sort == "Sport" or sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution, values='Number of Medals', path=['Sport', 'Country'], height=800, title= "Medals and sports for all countries")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800, title= "Medals and sports for all countries")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800, title= "Medals and sports for all countries")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800, title= "Medals and sports for all countries")
        return fig
    elif sort =="Country":   
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution, values='Number of Medals', path=['Country', 'Sport'], height=800, title= "Medals and sports for all countries")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800, title= "Medals and sports for all countries")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800, title= "Medals and sports for all countries")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800, title= "Medals and sports for all countries")
        
        return fig

@callback(
Output("graph_1_down_left", "figure"),
Input("year_dropdown", "value"),
Input("sport_dropdown", "value"),
Input("country_dropdown_down_left", "value")
)
def figure_three(years, sports, sort):
    if sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760, hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1.2, opacity=0.5,
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = participants_medals.query("Year==@years")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1,
            title="Size according to count of participants and Season")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800,width= 760, hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport_year.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
    elif sort =="Summer":
        if (years in [None, "", []]) and (sports in [None, "", []]):
            df = participants_medals_season_start
            df = df.query('Season == "Summer"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760, hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1.2, opacity=0.5,
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = participants_medals_season.query("Year==@years")
            df = df.query('Season == "Summer"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1,
            title="Size according to count of participants and Season")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_season.query("Sport==@sports")
            df  = df.query('Season == "Summer"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800,width= 760, hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_season.query("Year==@years")
            df = df.query("Sport==@sports")
            df  = df.query('Season == "Summer"')    
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
    elif sort =="Winter":
        if (years in [None, "", []]) and (sports in [None, "", []]):
            df = participants_medals_season_start
            df = df.query('Season == "Winter"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760, hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1.2, opacity=0.5,
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = participants_medals_season.query("Year==@years")
            df = df.query('Season == "Winter"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1,
            title="Size according to count of participants and Season")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_season.query("Sport==@sports")
            df = df.query('Season == "Winter"')
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800,width= 760, hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_season.query("Year==@years")
            df = df.query("Sport==@sports")
            df = df.query('Season == "Winter"')   
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants and Season")
            
    fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)
    
    fig.update_layout(                       
    mapbox_style="white-bg",
    mapbox_layers=[
    {
    "below": 'traces',
    "sourcetype": "raster",
    "sourceattribution": "United States Geological Survey",
    "source": [
    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
    ]
    }
    ])
    return fig

@callback(
    Output("graph_1_down_right", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value"),
    Input("country_dropdown_down_right", "value"),

)
def figure_four(years, sports, sort):
    if sort == "Sports" or sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution_sweden, values='Number of Medals', color_discrete_sequence=px.colors.qualitative.Pastel1, path=['Sport', 'Medal'], height=800, title = "Medals and sports for team SWEDEN")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution_sweden.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Medal'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution_sweden.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Medal'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution_sweden.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Medal'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        return fig
    elif sort =="Medals":   
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution_sweden, values='Number of Medals', path=['Medal', 'Sport'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution_sweden.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Medal', 'Sport'],color_discrete_sequence=px.colors.qualitative.Pastel1,height=800, title = "Medals and sports for team SWEDEN")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution_sweden.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Medal', 'Sport'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution_sweden.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Medal', 'Sport'],color_discrete_sequence=px.colors.qualitative.Pastel1, height=800, title = "Medals and sports for team SWEDEN")
        return fig
    

if __name__ == "__main__":
    app.run(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\yonis\.virtualenvs\OS-Project-17Ajirj2\Lib\site-packages\pandas\core\computation\scope.py:231, in Scope.resolve(
    self=Scope(scope=['Timestamp',
 'datetime',
 'True',
...de',
 'Continent_longitude',
 'Name',
 'Medal']
),
    key='Season',
    is_local=False
)
    230 if self.has_resolvers:
--> 231     return self.resolvers[key]
        self = Scope(scope=['Timestamp',
 'datetime',
 'True',
 'False',
 'list',
 'tuple',
 'inf',
 'Inf',
 '__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__builtin__',
 '__builtins__',
 '_ih',
 '_oh',
 '_dh',
 'In',
 'Out',
 'get_ipython',
 'exit',
 'quit',
 'open',
 '_',
 '__',
 '___',
 '__vsc_ipynb_file__',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'Dash',
 'html',
 'dcc',
 'callback',
 'Output',
 'Input',
 'dash_table',
 'dbc',
 'px',
 'pd',
 'load_figure_template',
 '_i2',
 'a

In [ ]:
px.sunburst(medal_distribution, values='Number of Medals', path=['Sport', 'Country'], 
            #Remove background color
            template='plotly_white')
            # )